<img src="images/cat.jpg">

### <font color=blue>Importing required libraries</font>

In [ ]:
import random
random.seed(7)

import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
import itertools
from PIL import Image
from scipy import ndimage
from lr_utils import load_dataset
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

%matplotlib inline

from warnings import filterwarnings
filterwarnings("ignore")

## <font color=blue>Overview of the Problem set</font>  ##

**Problem Statement**: You are given a dataset ("data.h5") containing:
    - a training set of m_train images labeled as cat (y=1) or non-cat (y=0)
    - a test set of m_test images labeled as cat or non-cat
    - each image is of shape (num_px, num_px, 3) where 3 is for the 3 channels (RGB). Thus, each image is square (height = num_px) and (width = num_px).


In [ ]:
# Loading the data (cat/non-cat)
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

In [ ]:
# Example of a picture
index = 97
plt.imshow(train_set_x_orig[index])
print ("y = " + str(train_set_y[:,index]) + ", it's a '" + classes[np.squeeze(train_set_y[:,index])].decode("utf-8") +  "' picture.")

Many software bugs in deep learning come from having matrix/vector dimensions that don't fit. If you can keep your matrix/vector dimensions straight you will go a long way toward eliminating many bugs. 

**Exercise:** Find the values for:
    - m_train (number of training examples)
    - m_test (number of test examples)
    - num_px (= height = width of a training image)
Remember that `train_set_x_orig` is a numpy-array of shape (m_train, num_px, num_px, 3). For instance, you can access `m_train` by writing `train_set_x_orig.shape[0]`.

In [ ]:

m_train = train_set_y.shape[1]
m_test = test_set_y.shape[1]
num_px = train_set_x_orig.shape[1]


print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/Width of each image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

For convenience, you should now reshape images of shape (num_px, num_px, 3) in a numpy-array of shape (num_px $*$ num_px $*$ 3, 1). After this, our training (and test) dataset is a numpy-array where each column represents a flattened image. There should be m_train (respectively m_test) columns.

In [ ]:

train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1)
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1)
train_set_y = train_set_y[0]
test_set_y = test_set_y[0]
print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print ("test_set_y shape: " + str(test_set_y.shape))
print ("sanity check after reshaping: " + str(train_set_x_flatten[0:5,0]))

### <font color=blue>Normalizing the data</font>

To represent color images, the red, green and blue channels (RGB) must be specified for each pixel, and so the pixel value is actually a vector of three numbers ranging from 0 to 255.

One common preprocessing step in machine learning is to center and standardize your dataset, meaning that you substract the mean of the whole numpy array from each example, and then divide each example by the standard deviation of the whole numpy array. But for picture datasets, it is simpler and more convenient and works almost as well to just divide every row of the dataset by 255 (the maximum value of a pixel channel).

<!-- During the training of your model, you're going to multiply weights and add biases to some initial inputs in order to observe neuron activations. Then you backpropogate with the gradients to train the model. But, it is extremely important for each feature to have a similar range such that our gradients don't explode. You will see that more in detail later in the lectures. !--> 

Let's standardize our dataset.

In [ ]:
train_set_x = train_set_x_flatten / 255.
test_set_x = test_set_x_flatten / 255.

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

### <font color=blue>Logistic Regression</font>

In [ ]:
from sklearn import linear_model
logistic = linear_model.LogisticRegression()
logistic.fit(train_set_x, train_set_y)
prediction = logistic.predict(test_set_x)
print(accuracy_score(test_set_y, prediction))

In [ ]:
cnf_matrix = confusion_matrix(test_set_y, prediction)
cnf_matrix

In [ ]:
plot_confusion_matrix(cnf_matrix, classes=['non-cat','cat'])

### <font color=blue>Support Vector Machine</font>

In [ ]:
from sklearn import svm
svm_clf = svm.SVC(kernel='linear')
svm_clf.fit(train_set_x, train_set_y)
prediction = svm_clf.predict(test_set_x)
print(accuracy_score(test_set_y, prediction))

In [ ]:
cnf_matrix = confusion_matrix(test_set_y, prediction)
cnf_matrix

In [ ]:
plot_confusion_matrix(cnf_matrix, classes=['non-cat','cat'])

### <font color=blue>Random Forest Tree</font>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rft_clf = RandomForestClassifier()
rft_clf.fit(train_set_x, train_set_y)
prediction = rft_clf.predict(test_set_x)
print(accuracy_score(test_set_y, prediction))

In [ ]:
cnf_matrix = confusion_matrix(test_set_y, prediction)
cnf_matrix

In [ ]:
plot_confusion_matrix(cnf_matrix, classes=['non-cat','cat'])

In [ ]:
import random
random.seed(7)

In [ ]:
rft_clf = RandomForestClassifier(n_estimators=100)
rft_clf.fit(train_set_x, train_set_y)
prediction = rft_clf.predict(test_set_x)
print(accuracy_score(test_set_y, prediction))

In [ ]:
cnf_matrix = confusion_matrix(test_set_y, prediction)
cnf_matrix

In [ ]:
plot_confusion_matrix(cnf_matrix, classes=['non-cat','cat'])

In [ ]:
rft_clf = RandomForestClassifier(n_estimators=200)
rft_clf.fit(train_set_x, train_set_y)
prediction = rft_clf.predict(test_set_x)
print(accuracy_score(test_set_y, prediction))

In [ ]:
cnf_matrix = confusion_matrix(test_set_y, prediction)
cnf_matrix

In [ ]:
plot_confusion_matrix(cnf_matrix, classes=['non-cat','cat'])